# Imports

In [1]:
from glob import glob
from pybaseball import playerid_reverse_lookup
import pandas as pd
import numpy as np

# Functions

In [2]:
def gather_hits(df, pitch_types, hit_types):
    """
    df : <Pandas DataFrame Object>
    pitch_types : <List> Contains all the columns for the different pitch types.
    hit_types : <List> Contains all of the columns to store the whether or not a hit occured
        for some pitch type.
    
    Removes pitches from the players record where the pitch resulted in a ball.
    Hit_type records whether or not the pitch for a specific pitch_type resulted in a hit.
    Foul balls are not considered to be a succesful hit.
    
    returns : <Pandas DataFrame Object> Updated DataFrame.
    """
    for pitch_type, hit_type in zip(pitch_types, hit_types):
        # Only count a pitch against a player if it wasn't a ball, because otherwise its the pitcher's fault.
        df[pitch_type] = ((df[pitch_type] == 1) & ( ~df["description"].str.contains("ball"))) * 1
        df[hit_type] = ((df[pitch_type] == 1) & (df["description"].str.contains("hit"))) * 1
        
    return df

def calculate_percentage_hit(df, pitch_types, hit_types):
    """
    df : <Pandas DataFrame Object>
    pitch_types : <List> Contains all the columns for the different pitch types.
    hit_types : <List> Contains all of the columns to store the whether or not a hit occured
        for some pitch type.
    
    Calculates percentage hit chance for some pitch type. (hit_type / pitch_type) * 100
    Removes NaN values by replacing them with zero. A NaN value my occur if a player never
        was pitched a pitch_type (0/0 error).
    
    returns : <Pandas DataFrame Object> Updated DataFrame.
    """
    for pitch_type, hit_type in zip(pitch_types, hit_types):
        df[pitch_type.replace("pitch_type", "percentage_hit")] = (df[hit_type] / df[pitch_type]) * 100
    
    df = df.fillna(0)
    return df

def calculate_specialty_score(df, percentage_hit_types, weights):
    """
    df : <Pandas DataFrame Object>
    percentage_hit_types : <List> The column names containing percentage score for each pitch type.
    weights : <Dict> The corresponding weights for each pitch type.
    
    Sums up all of the percentage hits and applies their corresponding weights to get a specialty score
        statistic.
    
    returns : <Pandas DataFrame Object> Updated DataFrame.
    """
    df["speciality_score"] = 0
    for percentage_hit_type in percentage_hit_types:
        df["speciality_score"] += df[percentage_hit_type] * weights[percentage_hit_type]
    
    return df

def load_data(data_path):
    """
    data_path : <str> Path containing csv files with each season's batting data. Ex: ./data/*
    
    returns : <Pandas DataFrame Object> DataFrame containing concatenated data across all seasons
        found within 'data_path'.
    """
    all_data = [pd.read_csv(file_name, index_col=0) for file_name in glob(data_path)]
    return pd.concat(all_data, axis=0, sort=False)

# Loading all Statcast data

In [3]:
data_path = "./data/*"
combined_data = load_data(data_path)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: DtypeWarning: Columns (46) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Data Processing

In [5]:
# Remove all instances where the pitch type was not recorded.
n = combined_data.dropna(axis=0, subset=["pitch_type"])

# Below removes random pitch types that consist of numbers that look like 1623_12312 and only occur once.
# This line wil give a warning. Just ignore it.
n["pitch_type"] = n["pitch_type"].apply(lambda x: x.replace("_",""))
n = n[~(n["pitch_type"].str.isdigit())]

# Add columns with a 1 column indicating pitch type.
n = pd.get_dummies(n, columns=["pitch_type"])

# Add column to sum to get number of plate appearances.
n["plate_appearance"] = 1

# Add player info to the batting data, matching on the batting id (key_mlbam).
n["key_mlbam"] = n["batter"]
player_info = playerid_reverse_lookup(n["batter"].values, key_type='mlbam')
n = pd.merge(n, player_info,on="key_mlbam", how="inner")
n["season"] = n["game_year"]

pitch_types = ['CH', 'CU', 'EP', 'FC', 'FF', 'FO', 'FS', 'FT', 'IN', 'KC', 'KN', 'PO', 'SC', 'SI', 'SL', 'UN']
pitch_types = ["pitch_type_" + pitch_type for pitch_type in pitch_types]
hit_types = [pitch_type.replace("pitch_type", "hit") for pitch_type in pitch_types]

z = gather_hits(n.copy(), pitch_types, hit_types)

pitch_type_aggregates = {pitch_type:"sum" for pitch_type in pitch_types}
hit_type_aggregates = {hit_type:"sum" for hit_type in hit_types}
aggregates = {"name_last":"first", "name_first":"first", "season":"first", "key_retro":"first", "key_bbref":"first", "plate_appearance":"sum", **pitch_type_aggregates, **hit_type_aggregates}

z = z.groupby(["batter", "game_year"]).agg(aggregates)

# Only get players with 200 or more plate appearances.
z = z[z["plate_appearance"] >= 200]

percentage_hit = calculate_percentage_hit(z.copy(), pitch_types, hit_types)

percentage_hit_types = [pitch_type.replace("pitch_type", "percentage_hit") for pitch_type in pitch_types]
weights = {'percentage_hit_CH': 1, 'percentage_hit_CU': 0, 'percentage_hit_EP': 1, 
           'percentage_hit_FC': 1, 'percentage_hit_FF': 1, 'percentage_hit_FO': 1, 
           'percentage_hit_FS': 1, 'percentage_hit_FT': 1, 'percentage_hit_IN': 0, 
           'percentage_hit_KC': 1, 'percentage_hit_KN': 1, 'percentage_hit_PO': 0, 
           'percentage_hit_SC': 1, 'percentage_hit_SI': 0, 'percentage_hit_SL': 1, 
           'percentage_hit_UN': 0}

specialty_score = calculate_specialty_score(percentage_hit.copy(), percentage_hit_types, weights)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Gathering player lookup table. This may take a moment.


In [6]:
specialty_score

name_last name_first  season key_retro  key_bbref  \
batter   game_year                                                      
110029.0 2014.0         abreu      bobby  2014.0  abreb001  abreubo01   
112526.0 2014.0         colon    bartolo  2014.0  colob001  colonba01   
         2016.0         colon    bartolo  2016.0  colob001  colonba01   
114739.0 2014.0        giambi      jason  2014.0  giamj001  giambja01   
116338.0 2014.0        hunter      torii  2014.0  huntt001  hunteto01   
         2015.0        hunter      torii  2015.0  huntt001  hunteto01   
116380.0 2014.0        ibanez       raul  2014.0  ibanr001  ibanera01   
116539.0 2014.0         jeter      derek  2014.0  jeted001  jeterde01   
117244.0 2014.0       konerko       paul  2014.0  konep001  konerpa01   
120074.0 2014.0         ortiz      david  2014.0  ortid001  ortizda01   
         2015.0         ortiz      david  2015.0  ortid001  ortizda01   
         2016.0         ortiz      david  2016.0  ortid001  ortizda01   
121347.0 2015.0     rodriguez       alex  2015.0  rodra001  rodrial01   
         2016.0     rodriguez       alex  2016.0  rodra001  rodrial01   
133380.0 2014.0       ramirez     aramis  2014.0  ramia001  ramirar01   
         2015.0       ramirez     aramis  2015.0  ramia001  ramirar01   
134181.0 2014.0        beltre     adrian  2014.0  belta001  beltrad01   
         2015.0        beltre     adrian  2015.0  belta001  beltrad01   
         2016.0        beltre     adrian  2016.0  belta001  beltrad01   
         2017.0        beltre     adrian  2017.0  belta001  beltrad01   
         2018.0        beltre     adrian  2018.0  belta001  beltrad01   
136767.0 2014.0        chavez       eric  2014.0  chave001  chaveer01   
136860.0 2014.0       beltran     carlos  2014.0  beltc001  beltrca01   
         2015.0       beltran     carlos  2015.0  beltc001  beltrca01   
         2016.0       beltran     carlos  2016.0  beltc001  beltrca01   
         2017.0       beltran     carlos  2017.0  beltc001  beltrca01   
150029.0 2014.0         werth     jayson  2014.0  wertj001  werthja01   
         2015.0         werth     jayson  2015.0  wertj001  werthja01   
         2016.0         werth     jayson  2016.0  wertj001  werthja01   
         2017.0         werth     jayson  2017.0  wertj001  werthja01   
...                       ...        ...     ...       ...        ...   
656669.0 2018.0        luplow     jordan  2018.0  luplj001  luplojo01   
656775.0 2018.0       mullins     cedric  2018.0  mullc002  mullice01   
656794.0 2018.0       newcomb       sean  2018.0  newcs001  newcose01   
656811.0 2018.0       o'hearn       ryan  2018.0  ohear001  ohearry01   
656941.0 2015.0     schwarber       kyle  2015.0  schwk001  schwaky01   
         2017.0     schwarber       kyle  2017.0  schwk001  schwaky01   
         2018.0     schwarber       kyle  2018.0  schwk001  schwaky01   
657077.0 2018.0       verdugo       alex  2018.0  verda001  verdual01   
657557.0 2017.0        dejong       paul  2017.0  dejop001  dejonpa01   
         2018.0        dejong       paul  2018.0  dejop001  dejonpa01   
657656.0 2018.0      laureano      ramon  2018.0  laurr001  laurera01   
660162.0 2017.0       moncada       yoan  2017.0  moncy001  moncayo01   
         2018.0       moncada       yoan  2018.0  moncy001  moncayo01   
660271.0 2018.0        ohtani     shohei  2018.0  ohtas001  ohtansh01   
660670.0 2018.0         acuna     ronald  2018.0  acunr001  acunaro01   
663656.0 2018.0        tucker       kyle  2018.0  tuckk001  tuckeky01   
664023.0 2017.0          happ        ian  2017.0  happi001   happia01   
         2018.0          happ        ian  2018.0  happi001   happia01   
664040.0 2018.0          lowe    brandon  2018.0  loweb001   lowebr01   
664056.0 2017.0         bader   harrison  2017.0  badeh001  baderha01   
         2018.0         bader   harrison  2018.0  badeh001  baderha01   
664057.0 2017.0     stevenson     andrew  2017.0  steva001  stevean01   
     

In [7]:
needed_stuff = specialty_score[["name_last", "name_first", "plate_appearance", "speciality_score"]]

In [8]:
needed_stuff.to_csv("2014-2018_specialty_score_with_ids.csv")

In [12]:
# specialty_score.to_csv("2014-2018_specialty_score_with_ids.csv")